# **Package installation**

In [1]:
pip install -U sentence-transformers

     |████████████████████████████████| 71kB 1.9MB/s 
     |████████████████████████████████| 1.1MB 7.5MB/s 
     |████████████████████████████████| 3.0MB 23.6MB/s 
     |████████████████████████████████| 890kB 40.1MB/s 
     |████████████████████████████████| 1.1MB 38.3MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-0.3.8-cp36-none-any.whl size=101996 sha256=92c47d5ac1dc9b8e1c6b7d6edc900aa7802b59151ecd191be10f18c6737546e5
  Stored in directory: /root/.cache/pip/wheels/27/ec/b3/d12cc8e4daf77846db6543033d3a5642f204c0320b15945647
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=33eb8aa9fb41955acd983e4b3b7e279b2c4ee563cf4c41851fc2e01c243a37c8
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sentence-transformers sacremoses


In [2]:
pip install gcld3

     |████████████████████████████████| 3.8MB 2.8MB/s 


In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import spacy
import pathlib
import random
import math
import re
import torch
from transformers import AutoTokenizer, AutoModel
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import euclidean_distances
from sentence_transformers import SentenceTransformer
from sklearn.feature_extraction.text import TfidfVectorizer

In [4]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
import gcld3

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
!ls '/content/drive/My Drive/Capstone Shared Docs/result'

'EC2 full run Oct14 result'  'Local Oct 13 result'
'EC2 Oct 13 result'	     'Local Oct 6 result'
'EC2 Oct26 public data'      'terminal log_oct5_am.txt'
'Error Log Resume.xlsx'      'terminal log_oct5.txt'


# **Loading dta files**

In [7]:
df_startup = pd.read_stata("drive/My Drive/Capstone Shared Docs/data/all_deals.dta")
df_startup_unique = df_startup.drop_duplicates(["portfoliocompanyid"],keep="first")

In [8]:
df_public = pd.read_stata("drive/My Drive/Capstone Shared Docs/data/all_public_firms.dta")
df_public_unique = df_public.drop_duplicates(["ÿþmark"],keep="first")

# **Loading txt Data Startups**

In [9]:
path_startup = pathlib.Path("drive/My Drive/Capstone Shared Docs/result/EC2 full run Oct14 result")

In [10]:
txt_startup =list(path_startup.glob('*.txt'))

In [11]:
all_text_startup = {}

for p in txt_startup:
    n = p.name.split("_")[0]
    file = open(p)
    # Remove first two and last characters, because of ascii encoding
    text = file.read().replace("***///***","")[2:-1]
    # remove ""
    text_list = text.split(" ")
    text_list = [x for x in text_list if x!=""]
    text = " ".join(text_list)
    file.close()
    all_text_startup[n] = text
  
print(len(all_text_startup))

430


In [12]:
Txt_startup_frame = pd.DataFrame.from_dict(all_text_startup, orient='index')
Txt_startup_frame.columns = ['Text']
Txt_startup_frame['companyid'] = Txt_startup_frame.index
Txt_startup_frame.index = np.arange(0,len(Txt_startup_frame.companyid))
Txt_startup_frame = Txt_startup_frame.assign(comp_type='S')
Txt_startup_frame = Txt_startup_frame[['companyid', 'comp_type', 'Text']]

In [13]:
stop_words_l=stopwords.words('english')
Txt_startup_frame['Text_cleaned']=Txt_startup_frame.Text.apply(lambda x: " ".join(re.sub(r'[^a-zA-Z]',' ',w).lower() for w in x.split() if re.sub(r'[^a-zA-Z]',' ',w).lower() not in stop_words_l) )

In [14]:
Txt_startup_frame.head()

,companyid,comp_type,Text,Text_cleaned
0,73501,S,JOOR Skip navigation Member Log In Forgot pass...,joor skip navigation member log forgot passwor...
1,160418,S,Patients Physicians Employers Login Crossover ...,patients physicians employers login crossover ...
2,161261,S,Home | Recent questions | Directories | Feedba...,home recent questions directories feedba...
3,129092,S,Nanotronics Home News Products What is nSpec ?...,nanotronics home news products nspec system ...
4,77179,S,Home About Us Our Approach Technology Our Cust...,home us approach technology customers contact ...


# **Loading txt Data Public Companies**

In [15]:
path_public = pathlib.Path("drive/My Drive/Capstone Shared Docs/result/EC2 Oct26 public data")

In [16]:
txt_public =list(path_public.glob('*.txt'))

In [17]:
all_text_public = {}

for p in txt_public:
    n = p.name.split("_")[0]
    file = open(p)
    # Remove first two and last characters, because of ascii encoding
    text = file.read().replace("***///***","")[2:-1]
    # remove ""
    text_list = text.split(" ")
    text_list = [x for x in text_list if x!=""]
    text = " ".join(text_list)
    file.close()
    all_text_public[n] = text

print(len(all_text_public))

536


In [18]:
Txt_public_frame = pd.DataFrame.from_dict(all_text_public, orient='index')
Txt_public_frame.columns = ['Text']
Txt_public_frame['companyid'] = Txt_public_frame.index
Txt_public_frame.index = np.arange(0,len(Txt_public_frame.companyid))
Txt_public_frame = Txt_public_frame.assign(comp_type='P')
Txt_public_frame = Txt_public_frame[['companyid', 'comp_type', 'Text']]

In [19]:
stop_words_l=stopwords.words('english')
Txt_public_frame['Text_cleaned']=Txt_public_frame.Text.apply(lambda x: " ".join(re.sub(r'[^a-zA-Z]',' ',w).lower() for w in x.split() if re.sub(r'[^a-zA-Z]',' ',w).lower() not in stop_words_l) )

In [20]:
Txt_public_frame
Txt_public_frame.head()

,companyid,comp_type,Text,Text_cleaned
0,151,P,CVR Energy is an independent petroleum refiner...,cvr energy independent petroleum refiner marke...
1,1658,P,COMPANY Executive Profiles History Company Val...,company executive profiles history company val...
2,5503,P,Home | About Us | Locations | ATM Locations | ...,home us locations atm locations contac...
3,2901,P,Home | Site Map | Contact Us Information is be...,home site map contact us information colle...
4,1510,P,Search: Corporate Info | Contact GAF | News & ...,search corporate info contact gaf news ...


# **Concatenate Public and Startup DB**

In [21]:
Txt_all_frame = pd.concat([Txt_startup_frame, Txt_public_frame])
Txt_all_frame.index = np.arange(0,len(Txt_all_frame.companyid))

In [22]:
Txt_all_frame.head()

,companyid,comp_type,Text,Text_cleaned
0,73501,S,JOOR Skip navigation Member Log In Forgot pass...,joor skip navigation member log forgot passwor...
1,160418,S,Patients Physicians Employers Login Crossover ...,patients physicians employers login crossover ...
2,161261,S,Home | Recent questions | Directories | Feedba...,home recent questions directories feedba...
3,129092,S,Nanotronics Home News Products What is nSpec ?...,nanotronics home news products nspec system ...
4,77179,S,Home About Us Our Approach Technology Our Cust...,home us approach technology customers contact ...


# **Solving some language issues**

In [23]:
Txt_all_frame.iloc[33:34,:]

,companyid,comp_type,Text,Text_cleaned
33,230198,S,Presentacin Asesores / Colaboradores Com desca...,presentacin asesores colaboradores com desca...


In [24]:
lan = []
rel_lan = []
detector = gcld3.NNetLanguageIdentifier(min_num_bytes=0, max_num_bytes=10000)

for i in range(len(Txt_all_frame['Text_cleaned'])):
  t =Txt_all_frame.Text_cleaned[i]
  t = str(t)
  
  result = detector.FindLanguage(text=t)
  lan.append(result.language)
  rel_lan.append(result.is_reliable)


In [25]:
Txt_all_frame['language'] = lan
Txt_all_frame['Rela_language'] = rel_lan

In [26]:
Txt_all_frame.language.unique()

array(['en', 'it', 'es', 'fr', 'mg', 'no', 'sn', 'gl', 'zu', 'eo', 'da',
       'fy', 'lb', 'de', 'sr', 'nl', 'fi', 'vi', 'hu', 'su', 'jv', 'pt',
       'ru-Latn'], dtype=object)

In [27]:
Txt_all_frame[Txt_all_frame.language=='es']

,companyid,comp_type,Text,Text_cleaned,language,Rela_language
27,112162,S,Bienvenido al Nuevo Mundo de Nativo Network . ...,bienvenido al nuevo mundo de nativo network ...,es,True
33,230198,S,Presentacin Asesores / Colaboradores Com desca...,presentacin asesores colaboradores com desca...,es,True
74,297244,S,movista.com Algunos sitios relacionados con: m...,movista com algunos sitios relacionados con m...,es,True
355,150133,S,Error 503 Service Unavailable Service Unavaila...,error service unavailable service unavaila...,es,True
508,4691,P,Call Us 24/7 1-800-786-7235 Why Bank Stem Cell...,call us bank stem cells v...,es,True
773,4640,P,Welcome to the Worlds Healthy Coffee Company M...,welcome worlds healthy coffee company manufact...,es,True
803,51,P,-------- Including dojo Configurations -------...,including dojo configurations ...,es,False


In [28]:
Txt_all_frame[Txt_all_frame.language=='fr']

,companyid,comp_type,Text,Text_cleaned,language,Rela_language
55,93431,S,banned interdit verboden vietato prohibido ver...,banned interdit verboden vietato prohibido ver...,fr,False
257,139205,S,Recherche de noms de domaine : www.indom.com L...,recherche de noms de domaine www indom com l...,fr,True
316,107417,S,Index of / cgi-bin/ dev/ Apache/2.2.15 (Unix) ...,index cgi bin dev apache unix mod...,fr,True
343,129019,S,banned interdit verboden vietato prohibido ver...,banned interdit verboden vietato prohibido ver...,fr,False
491,5208,P,Alpha Innotech is now part of Cell Biosciences...,alpha innotech part cell biosciences visit ww...,fr,True
650,5107,P,Forbidden You don't have permission to access ...,forbidden don t permission access server ap...,fr,True
733,3106,P,Skip Navigation Country UK (English) France (F...,skip navigation country uk english france f...,fr,True


In [29]:
Txt_all_frame_en = Txt_all_frame[(Txt_all_frame.language=='en') ] #| ( (Txt_all_frame.language!='en') & (Txt_all_frame.language==False) )
Txt_all_frame_en.index = np.arange(0,len(Txt_all_frame_en.companyid))

# **Lenght of text**

In [ ]:
word_len = [len(i.split()) for i in Txt_all_frame_en.Text_cleaned ]
text_len = [len(i) for i in Txt_all_frame_en.Text_cleaned ]
Txt_all_frame_en['Word_len'] = word_len
Txt_all_frame_en['Text_len'] = text_len

In [31]:
Txt_all_frame_en.head()

,companyid,comp_type,Text,Text_cleaned,language,Rela_language,Word_len,Text_len
0,73501,S,JOOR Skip navigation Member Log In Forgot pass...,joor skip navigation member log forgot passwor...,en,True,3614,27748
1,160418,S,Patients Physicians Employers Login Crossover ...,patients physicians employers login crossover ...,en,True,66,489
2,161261,S,Home | Recent questions | Directories | Feedba...,home recent questions directories feedba...,en,True,92,656
3,129092,S,Nanotronics Home News Products What is nSpec ?...,nanotronics home news products nspec system ...,en,True,2337,19264
4,77179,S,Home About Us Our Approach Technology Our Cust...,home us approach technology customers contact ...,en,True,167,1326


In [32]:
Txt_all_frame_en.Word_len.mean()

30677.095135135136

In [33]:
Txt_all_frame_en.Text_len.mean()

133178.79027027026

In [34]:
Txt_all_frame_en.Word_len.max()

4806695

In [35]:
Txt_all_frame_en.Text_len.max()

16065268

In [36]:
np.percentile(Txt_all_frame_en.Word_len, 95)

58185.59999999997

In [37]:
np.percentile(Txt_all_frame_en.Text_len, 95)

353595.39999999985

In [38]:
np.percentile(Txt_all_frame_en.Word_len, 99)

565369.6799999988

In [39]:
np.percentile(Txt_all_frame_en.Text_len, 99)

2461761.719999997

# **TF-IDF**

In [41]:
stopWords = set(stopwords.words("english"))

In [42]:
vectorizer = TfidfVectorizer(stop_words = stopWords)

In [43]:
Complete_frame = Txt_all_frame_en
Complete_frame.head(9)

,companyid,comp_type,Text,Text_cleaned,language,Rela_language,Word_len,Text_len
0,73501,S,JOOR Skip navigation Member Log In Forgot pass...,joor skip navigation member log forgot passwor...,en,True,3614,27748
1,160418,S,Patients Physicians Employers Login Crossover ...,patients physicians employers login crossover ...,en,True,66,489
2,161261,S,Home | Recent questions | Directories | Feedba...,home recent questions directories feedba...,en,True,92,656
3,129092,S,Nanotronics Home News Products What is nSpec ?...,nanotronics home news products nspec system ...,en,True,2337,19264
4,77179,S,Home About Us Our Approach Technology Our Cust...,home us approach technology customers contact ...,en,True,167,1326
5,89064,S,StyleSaint Something incredible is happening i...,stylesaint something incredible happening fash...,en,True,8,68
6,76857,S,Client Login Signup Free Consultation Services...,client login signup free consultation services...,en,True,6487,49653
7,77461,S,Connection Engine beta Have an account? Sign i...,connection engine beta account sign username ...,en,True,107,873
8,88411,S,Blog Features About Management Team Investors ...,blog features management team investors adviso...,en,True,4664,35884


In [48]:
X_train = vectorizer.fit_transform(Complete_frame['Text_cleaned'])
X_train.shape

(925, 679401)

In [55]:
with open('/content/drive/My Drive/Capstone Shared Docs/TF_IDF_matrix.npy', 'wb') as f:
  np.save(f, X_train)

In [50]:
pairwise_similarities=cosine_similarity(X_train)

(925, 925)

In [51]:
def get_most_similar(sim_matrix_v, startup_index, how_many=1):
  sim_matrix_copy = sim_matrix_v.copy()
  v = sim_matrix_copy[startup_index,:]
  v[startup_index] = 0
  most_similar_indices = []
  similarity_list = []
  for i in range(how_many):
    ind = np.argmax(v)
    similarity = np.max(v)
    v[ind] = 0
    most_similar_indices.append(ind)
    similarity_list.append(similarity)
  return most_similar_indices, similarity_list

In [52]:
Dict_similarity = []
Strat_score = []
n = 5

for i in range(len(Complete_frame.companyid)):

  most_similar_index,most_similar_similarity = get_most_similar(pairwise_similarities, i, n)
  IDs = [Complete_frame.companyid[j] for j in most_similar_index]
  dict_sim = {IDs[i]: most_similar_similarity[i] for i in range(len(IDs))} 
  strategy_score = np.mean(1-np.array(most_similar_similarity))

  Dict_similarity.append(dict_sim)
  Strat_score.append(strategy_score)

In [ ]:
Complete_frame['Dict_similarity'] = Dict_similarity
Complete_frame['Strat_score'] = Strat_score

In [56]:
Complete_frame.to_csv('/content/drive/My Drive/Capstone Shared Docs/TF_IDF_Results.csv')

In [54]:
Complete_frame.head()

,companyid,comp_type,Text,Text_cleaned,language,Rela_language,Word_len,Text_len,Dict_similarity,Strat_score
0,73501,S,JOOR Skip navigation Member Log In Forgot pass...,joor skip navigation member log forgot passwor...,en,True,3614,27748,"{'96698': 0.11229419254224686, '180156': 0.112...",0.890763
1,160418,S,Patients Physicians Employers Login Crossover ...,patients physicians employers login crossover ...,en,True,66,489,"{'96769': 0.15112988169856748, '2626': 0.13765...",0.874485
2,161261,S,Home | Recent questions | Directories | Feedba...,home recent questions directories feedba...,en,True,92,656,"{'4178': 0.38381624059169467, '96769': 0.12201...",0.846288
3,129092,S,Nanotronics Home News Products What is nSpec ?...,nanotronics home news products nspec system ...,en,True,2337,19264,"{'6238': 0.09628984286210562, '3322': 0.082600...",0.923961
4,77179,S,Home About Us Our Approach Technology Our Cust...,home us approach technology customers contact ...,en,True,167,1326,"{'1690': 0.07245344750386438, '240117': 0.0720...",0.930001
